<a href="https://colab.research.google.com/github/kazu-gor/nlp/blob/main/basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module

In [ ]:
!apt-get -q -y install swig 
!apt-get install mecab
!apt-get install libmecab-dev
!apt-get install mecab-ipadic-utf8
!pip install mecab-python3==0.996.5
!pip install unidic-lite

In [ ]:
import MeCab
import numpy as np
import time

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 相互情報量 (Pointwise Mutual Information)

出現回数を考慮することで，単語の関係性を適切に抽出することができる．
<br><br>
${PMI(x, y) = \log _2\frac{P(x, y)}{P(x)P(y)}}$
<br><br>
共起行列を${C}$，出現回数を${N}$とすると以下のような式になる．　

${PMI(x, y) = \log _2\frac{C(x, y)\cdot N}{C(x)C(y)}}$
<br><br>
この状態での問題点は，共起する列が0の場合， ${\log_2 0=-\infty}$となってしまうため，<b>正の相互情報量<b>(Positive PMI)が使われる．
<br><br>
${PPMI(x, y) = \max(0, PMI(x, y))}$



In [ ]:
def ppmi(C: np.array, verbose=False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[i] * S[j]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100) == 0:
                    print('%.1f%% done' % (100*cnt / total))
    return M

In [ ]:
# test code
C = np.zeros((5, 5))
C[0][0], C[0][3], C[0][4], C[2][0], C[2][2], C[2][4], C[3][3], C[4, 4] = 1, 1, 1, 1, 1, 1, 1, 1
print(f"covariance matrix \n{C}")
print(f"PPMI\n{ppmi(C)}")

# 次元削除

共分散行列および，相互情報量はまだ疎な行列であるため，情報量が増えた場合，計算量が比例して増えてしまう．
<br><br>
これを回避するために，次元削除を行い，重要な情報だけ抽出して引き出せばよい．
<br><br>
その一種として<b>特異値分解(Singular Value Decomposition: SVD)</b>を使う．

${X = USV^T}$

任意の行列 ${X}$ を, ${U, S, V}$の3つの行列に分解する．だたし，${U, V}$は直行行列であり， ${S}$は対角行列である．　

${U}$ は「単語空間」であり，　${S}$の対角成分は「特異値(重要度)」というものが大きい順にならんでいる．

In [ ]:
W = ppmi(C)

#SVD
U, S, V = np.linalg.svd(W)

In [ ]:
# covariance matrix
print(C[0])

# PPMI matrix
print(W[0])

#SVD
print(U[0])